In [41]:
import pandas as pd

In [42]:
Dataset=pd.read_csv('CKD.csv')

In [43]:
Dataset.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hrmo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [44]:
Dataset.shape

(399, 25)

In [45]:
Dataset.info

<bound method DataFrame.info of            age         bp sg  al  su     rbc        pc         pcc  \
0     2.000000  76.459948  c   3   0  normal  abnormal  notpresent   
1     3.000000  76.459948  c   2   0  normal    normal  notpresent   
2     4.000000  76.459948  a   1   0  normal    normal  notpresent   
3     5.000000  76.459948  d   1   0  normal    normal  notpresent   
4     5.000000  50.000000  c   0   0  normal    normal  notpresent   
..         ...        ... ..  ..  ..     ...       ...         ...   
394  51.492308  70.000000  a   0   0  normal    normal  notpresent   
395  51.492308  70.000000  c   0   2  normal    normal  notpresent   
396  51.492308  70.000000  c   3   0  normal    normal  notpresent   
397  51.492308  90.000000  a   0   0  normal    normal  notpresent   
398  51.492308  80.000000  a   0   0  normal    normal  notpresent   

             ba         bgr  ...        pcv            wc        rc  htn   dm  \
0    notpresent  148.112676  ...  38.868902   

In [46]:
Dataset=pd.get_dummies(Dataset,dtype=int,drop_first=True)

In [47]:
Dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [48]:
Dataset.info

<bound method DataFrame.info of            age         bp  al  su         bgr          bu        sc  \
0     2.000000  76.459948   3   0  148.112676   57.482105  3.077356   
1     3.000000  76.459948   2   0  148.112676   22.000000  0.700000   
2     4.000000  76.459948   1   0   99.000000   23.000000  0.600000   
3     5.000000  76.459948   1   0  148.112676   16.000000  0.700000   
4     5.000000  50.000000   0   0  148.112676   25.000000  0.600000   
..         ...        ...  ..  ..         ...         ...       ...   
394  51.492308  70.000000   0   0  219.000000   36.000000  1.300000   
395  51.492308  70.000000   0   2  220.000000   68.000000  2.800000   
396  51.492308  70.000000   3   0  110.000000  115.000000  6.000000   
397  51.492308  90.000000   0   0  207.000000   80.000000  6.800000   
398  51.492308  80.000000   0   0  100.000000   49.000000  1.000000   

            sod       pot       hrmo  ...  pc_normal  pcc_present  ba_present  \
0    137.528754  4.627244  12.5181

In [49]:
Independent=Dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
Dependent=Dataset[['classification_yes']]

In [50]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [51]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
Y_train=sc.fit_transform(Y_train)

In [58]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

param_grid = {
    'alpha': [0.0, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
    'fit_prior': [True, False]
    
}
grid = GridSearchCV(MultinomialNB(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


ValueError: 
All the 70 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
14 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\naive_bayes.py", line 736, in fit
    Y = labelbin.fit_transform(y)
        ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix()